# Identifying Denver, Colorado Neighborhoods Suitable for a New Brewery Venue
## IBM Applied Data Science Capstone Project
## Introduction
According to the Denver Microbrew Tour, the City of Denver has 92 brewery venues as of early 2020. In a city spanning a modest area of 401 km2, there exists approximately one brewery for every 4 km2. Home to North America’s annual Great American Beer Festival and listed as the number one destination for craft beer tourism on Wikipedia, it appears Denver’s local economy is willing to accommodate for such a high density of craft beer venues. 
This project leverages venue geolocation and machine learning clustering algorithm to help small business entrepreneurs identify which neighborhoods in Denver, Colorado have the greatest potential for being the home of the city’s next craft brewery.
## Data
The data used for this project are a list of 51 neighborhoods in Denver, Colorado, obtained by scraping the “Neighborhoods in Denver” category from Wikipedia using the BeautifulSoup python package. Once the list of neighborhoods was read into a data table, each neighborhoods’ geographic information was georeferenced using ESRI’s ArcGIS geocoder. Then, using Foursquare’s API, a search request returned the top 100 venues within 4 km of each neighborhood’s centroid. Note that a radius of 4 km was chosen based on the 4 km2 density of Denver breweries. The dataset returned by Foursquare containing the top 100 venues was then sorted by neighborhood and venue type, and lastly the top 10 most occurring venues per neighborhood were selected. A total of 33 neighborhoods out of 51 total had a brewery listed as one of the top 10 most occurring venues.

##### Import libraries for project

In [1]:
import pandas as pd

# stops pandas warnings
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup

import folium
from geopy.geocoders import ArcGIS

import requests

import numpy as np

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

##### Create a DataFrame of Denver, Colorado Neighborhoods from a list scraped from Wikipedia

In [2]:
# read Wikipedia webpage containing Denver neighborhoods list
data = requests.get('https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Denver').text

# initialize BeautifulSoup
soup = BeautifulSoup(data, 'html.parser')

# initialize an object to write the list to
neighborhoods = []

# write neighborhoods to neighborhood list
for i in soup.find_all('div', class_='mw-category')[0].findAll('li'):
    neighborhoods.append(i.text)

# create a pandas DataFrame of Denver neighborhoods    
den_df = pd.DataFrame({'Neighborhood' : neighborhoods}).iloc[1:]
den_df['Neighborhood'] = den_df['Neighborhood'].str.split(pat=', ', expand=True)[0]

# ensure that the DataFrame is the correct shape and check the format
print(den_df.shape)
den_df.head()

(51, 1)


,Neighborhood
1,Alamo Placita
2,Athmar Park
3,Auraria
4,Baker
5,Barnum


##### Add geocoded latitude and longitude columns to the neighborhoods in the Denver DataFrame

In [3]:
# initialize lat and log columns for DataFrame
den_df['Latitude'] = ''
den_df['Longitude'] = ''

# initalize the ESRI ArcGIS geocoder
geolocator = ArcGIS(user_agent="my-denver")

# geolocate neighborhoods using the ESRI ArcGIS geocoder
for i in range(0,len(den_df['Neighborhood'])):
    address = den_df['Neighborhood'].iloc[i] + ', Denver, Colorado'
    location = geolocator.geocode(address)
    den_df['Latitude'].iloc[i] = location.latitude
    den_df['Longitude'].iloc[i] = location.longitude

# show results for all neighborhoods    
den_df

,Neighborhood,Latitude,Longitude
1,Alamo Placita,39.7211,-104.975
2,Athmar Park,39.704,-105.01
3,Auraria,39.7458,-105.01
4,Baker,39.7112,-104.992
5,Barnum,39.7182,-105.033
6,Belcaro,39.7038,-104.95
7,Berkeley,39.7767,-105.04
8,Capitol Hill,39.7337,-104.98
9,Central Park,39.8924,-104.952
10,Chaffee Park,39.7874,-105.018


##### Build a map showing geolocated Denver neighborhoods

In [4]:
# locate the center of Denver to center the map
address = 'Denver'

geolocator = ArcGIS(user_agent="my-denver")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

den_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(den_df['Latitude'], den_df['Longitude'], den_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(den_map)  

# display the map
den_map

##### Save the map

In [5]:
# save map as HTML
den_map.save('den_neighborhoods.html')

##### Store Foursquare credientials

In [25]:
CLIENT_ID = 'your Foursquare ID'  # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


##### Define a function to GET venues from Foursquare

In [7]:
# define a function to retireve the top 100 venues within a 4000m radius from a neighborhood's centroid
def getNearbyVenues(names, latitudes, longitudes, radius=4000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Request top 100 venues within a 4000m radius of each Denver neighborhood

In [8]:
# GET top 100 venues within 1000m of the centriod of each Denver neighborhood
den_vens = getNearbyVenues(names=den_df['Neighborhood'], latitudes=den_df['Latitude'], longitudes=den_df['Longitude'])
den_vens.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alamo Placita,39.72107,-104.97522,Table 6,39.725824,-104.974280,American Restaurant
1,Alamo Placita,39.72107,-104.97522,Trader Joe's,39.726880,-104.982545,Grocery Store
2,Alamo Placita,39.72107,-104.97522,Carmine's on Penn,39.714959,-104.981310,Italian Restaurant
3,Alamo Placita,39.72107,-104.97522,Cosmo's Pizza,39.725069,-104.983255,Pizza Place
4,Alamo Placita,39.72107,-104.97522,Sweet Action Ice Cream,39.717544,-104.987352,Ice Cream Shop


##### See how many venues were returned for each neighborhood

In [9]:
# list the total number of venues found for each neighborhood
den_vens.groupby(['Neighborhood']).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alamo Placita,100,100,100,100,100,100
Athmar Park,100,100,100,100,100,100
Auraria,100,100,100,100,100,100
Baker,100,100,100,100,100,100
Barnum,100,100,100,100,100,100
Belcaro,100,100,100,100,100,100
Berkeley,100,100,100,100,100,100
Capitol Hill,100,100,100,100,100,100
Central Park,100,100,100,100,100,100


##### Inspect the unique venues returned by the Foursquare GET request

In [10]:
# list all unique venue types found in GET request
den_vens['Venue Category'].unique()

array(['American Restaurant', 'Grocery Store', 'Italian Restaurant',
       'Pizza Place', 'Ice Cream Shop', 'Brewery', 'Indie Movie Theater',
       'Bookstore', 'Coffee Shop', 'Bagel Shop', 'Yoga Studio',
       'Breakfast Spot', 'Cajun / Creole Restaurant', 'Sandwich Place',
       'Clothing Store', 'Salon / Barbershop', 'Park',
       'Thrift / Vintage Store', 'Hotel', 'Japanese Restaurant',
       'Burger Joint', 'Tailor Shop', 'Botanical Garden', 'Donut Shop',
       'Bar', 'French Restaurant', 'Arts & Crafts Store', 'Trail',
       'Ski Shop', 'Thai Restaurant', 'Taco Place', 'Cocktail Bar',
       'Marijuana Dispensary', 'Chinese Restaurant',
       'New American Restaurant', 'Creperie', 'Art Museum', 'Juice Bar',
       'Liquor Store', 'History Museum', 'Cosmetics Shop', 'Garden',
       'Vegetarian / Vegan Restaurant', 'Distillery', 'Cuban Restaurant',
       'Furniture / Home Store', 'Concert Hall', 'Arts & Entertainment',
       'Mexican Restaurant', 'Spa', 'Museum', 'Smoke

##### Ensure that Brewery is one of the unique listed venue types

In [11]:
# check to see if Brewery is a listed venue type
'Brewery' in den_vens['Venue Category'].unique()

True

##### Begin the clustering analysis of Denver neighborhoods by creating a one hot encoded dataframe that is ready for clustering

In [12]:
# one hot encoding
den_onehot = pd.get_dummies(den_vens[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
den_onehot['Neighborhood'] = den_vens['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [den_onehot.columns[-1]] + list(den_onehot.columns[:-1])
den_onehot = den_onehot[fixed_columns]

print(den_onehot.shape)
den_onehot.head()

(5080, 234)


,Neighborhood,ATM,Accessories Store,Adult Education Center,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,...,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Alamo Placita,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alamo Placita,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Alamo Placita,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alamo Placita,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Alamo Placita,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Group Denver neighborhoods with the same venue types

In [13]:
den_grouped = den_onehot.groupby('Neighborhood').mean().reset_index()
print(den_grouped.shape)
den_grouped.head()

(51, 234)


,Neighborhood,ATM,Accessories Store,Adult Education Center,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,...,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Alamo Placita,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.02,0.0,0.0
1,Athmar Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.04,0.01,0.00,0.00,0.0,0.0,0.02,0.0,0.0
2,Auraria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.01,0.00,0.0,0.0,0.02,0.0,0.0
3,Baker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.04,0.00,0.00,0.00,0.0,0.0,0.02,0.0,0.0
4,Barnum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.05,0.01,0.00,0.01,0.0,0.0,0.02,0.0,0.0


##### Define a function to sort venues by occurence

In [14]:
# function pulls top x venues for each row in the onehot encoded data by frequency of occurence
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Create a dataframe with the top ten venues of each neighborhood group

In [15]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = den_grouped['Neighborhood']

for ind in np.arange(den_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(den_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alamo Placita,American Restaurant,Coffee Shop,Park,Sandwich Place,Pizza Place,Ice Cream Shop,Italian Restaurant,Brewery,Mexican Restaurant,Burger Joint
1,Athmar Park,Mexican Restaurant,Brewery,Coffee Shop,Vietnamese Restaurant,Pizza Place,Park,Marijuana Dispensary,Bar,Sandwich Place,Bakery
2,Auraria,Hotel,Brewery,Park,New American Restaurant,Italian Restaurant,Mexican Restaurant,American Restaurant,Pizza Place,Lounge,Coffee Shop
3,Baker,Coffee Shop,Mexican Restaurant,Brewery,Vietnamese Restaurant,Pizza Place,American Restaurant,Marijuana Dispensary,Italian Restaurant,Park,French Restaurant
4,Barnum,Mexican Restaurant,Brewery,Coffee Shop,Vietnamese Restaurant,Spa,Ice Cream Shop,Marijuana Dispensary,American Restaurant,Bar,Donut Shop


##### Find Denver neighborhoods with a brewery as the top 10 occuring venues

In [16]:
# identify Denver neighborhoods with a Brewery in their top 10 venues
den_merged = neighborhoods_venues_sorted.loc[np.where(neighborhoods_venues_sorted == 'Brewery')[0]]

# merge neighborhood venues with den_df to add latitude/longitude for each neighborhood
den_merged = den_merged.join(den_df.set_index('Neighborhood'), on='Neighborhood')

print(len(den_merged['Neighborhood']))
den_merged['Neighborhood'] # check the last columns!

33


0             Alamo Placita
1               Athmar Park
2                   Auraria
3                     Baker
4                    Barnum
6                  Berkeley
7              Capitol Hill
9              Chaffee Park
12                Chinatown
14           City Park West
15             Civic Center
16                  Clayton
17                     Cole
20          Downtown Denver
21           Elyria-Swansea
22               Globeville
25                 Highland
27           Jefferson Park
28             Lincoln Park
29                     LoDo
33       North Capitol Hill
35                    Regis
36                Ruby Hill
37    Sloan Lake (Colorado)
38                    Speer
40               Sun Valley
41                Sunnyside
42            Union Station
43               University
44                 Valverde
45               Villa Park
49              West Colfax
50                 Whittier
Name: Neighborhood, dtype: object

##### Map Denver neighborhoods with a brewery as the top 10 occuring venues

In [17]:
# locate the center of Denver to center the map
address = 'Denver'

geolocator = ArcGIS(user_agent="my-denver")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

den_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(den_merged['Latitude'], den_merged['Longitude'], den_merged['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(den_map)  
    
den_map

##### Save the map

In [18]:
# save map as HTML
den_map.save('den_breweries.html')

## Methodology
To identify neighborhoods estimated to be a good potential host for a new brewery site the machine learning algorithm k-means clustering from the scikit-learn python package was used. Given a user defined number of clusters to group the dataset, k-means clustering attempts to group input variables by mean distance from data centroids to minimize inertia. In order to avoid overfitting or underfitting of the data by grouping the dataset into too few or too many clusters, k-means clustering was run multiple times using a minimum of 3 clusters and a maximum of 20 clusters. Data for each run of k-means iteration was then output to a table in order to identify which cluster contained the most breweries, the total number of neighborhoods per cluster, and the number of neighborhoods. This information helped to identify the most desired outcome of the k-means model run.

In [19]:
# set min and max number of clusters
min_kclusters = 3
max_kclusters = 20

# prepare the clustering dataset
den_grouped_clustering = den_grouped.drop('Neighborhood', 1)

# merge neighborhood venues with den_df to add latitude/longitude for each neighborhood
den_merged = pd.merge(neighborhoods_venues_sorted, den_df, how='inner', on='Neighborhood')

# run k-means clustering build a table that lists the results for each number of clusters
print('N Clusters\t|Mode Cluster\t|Total Neighborhoods\t|Neighborhoods Without Breweries')
print('----------------------------------------------------------------------------------------')

for i in range(min_kclusters, (max_kclusters+1)):
    # run k-means clustering
    kmeans = KMeans(n_clusters=i, random_state=82).fit(den_grouped_clustering)
    # check cluster labels generated for each row in the dataframe
    den_merged['Cluster Labels {}'.format(i)] = kmeans.labels_
    # identify the cluster containing the most breweries
    mode_cluster = den_merged.loc[np.where(den_merged == 'Brewery')[0]]['Cluster Labels {}'.format(i)].mode()[0]
    # create a DataFrame for the cluster containing the most breweries
    brewery_cluster = den_merged.iloc[np.where(den_merged['Cluster Labels {}'.format(i)] == mode_cluster)]
    # print outputs
    print(i,'\t\t|', mode_cluster, '\t\t|', len(brewery_cluster), '\t\t\t|', len(brewery_cluster[brewery_cluster != 'Brewery'].dropna()))

N Clusters	|Mode Cluster	|Total Neighborhoods	|Neighborhoods Without Breweries
----------------------------------------------------------------------------------------
3 		| 2 		| 18 			| 0
4 		| 3 		| 13 			| 1
5 		| 4 		| 12 			| 1
6 		| 1 		| 12 			| 0
7 		| 2 		| 11 			| 0
8 		| 4 		| 10 			| 0
9 		| 3 		| 6 			| 0
10 		| 4 		| 7 			| 0
11 		| 2 		| 7 			| 0
12 		| 2 		| 7 			| 0
13 		| 2 		| 7 			| 0
14 		| 2 		| 7 			| 0
15 		| 2 		| 7 			| 0
16 		| 0 		| 7 			| 0
17 		| 3 		| 6 			| 0
18 		| 3 		| 6 			| 0
19 		| 0 		| 7 			| 0
20 		| 2 		| 5 			| 0


## Results
The k-means clustering algorithm with clusters ranging between a minimum of 3 and maximum number of 20 returned between 18 and 5 total neighborhoods, respectively, and each being the maximum or minimum number of total neighborhoods returned. K-means clustering outputs using 4 and 5 clusters where the only outputs to return a neighborhood that did not have brewery venue as a top ten most occurring venue. Given that the k-means run with 4 clusters contained 13 neighborhoods, one more than the run with 5 clusters, this output was chosen as the desired model outcome given the larger sample size. The one neighborhood identified as no having a brewery as a top ten most occurring venue was Denver’s Central Park neighborhood. It is important to note that the one neighborhood without a top ten most common brewery venue in the k-means clusters = 5 output was not Central Park, but instead the Montbello neighborhood.

In [20]:
# mode cluster for the desired k-means clustering outcome
mode_cluster = 3
# create a DataFrame for the cluster containing the most breweries
brewery_cluster = den_merged.iloc[np.where(den_merged['Cluster Labels 4'] == mode_cluster)]
# identify neighborhoods that do not have a brewery
potential_neighborhoods = brewery_cluster[brewery_cluster != 'Brewery'].dropna()
potential_neighborhoods.iloc[:, :11]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Park,Mexican Restaurant,Italian Restaurant,Pizza Place,Grocery Store,American Restaurant,Fast Food Restaurant,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Gym


##### Map the clustered Denver neighborhoods from the kmeans clustering = 4

In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
kclusters = 16

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(den_merged['Latitude'], den_merged['Longitude'], den_merged['Neighborhood'], den_merged['Cluster Labels 4']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Save the map

In [22]:
# save map as HTML
den_map.save('den_clusters.html')

##### Map neighborhoods from the breweries cluster that do not have a brewery as the top 10 most occuring venues. These are the neighborhoods where a new brewery would be recommended to open.

In [23]:
# locate the center of Denver to center the map
address = 'Denver'

geolocator = ArcGIS(user_agent="my-denver")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

den_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(potential_neighborhoods['Latitude'], potential_neighborhoods['Longitude'], potential_neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(den_map)  
    
den_map

##### Save the map

In [24]:
# save map as HTML
den_map.save('den_potential_neighborhoods.html')

## Discussion
This assignment felt pressuring in the sense that k-means clustering was the appropriate model to use for the situation and that Foursquare venue occurrence was an acceptable variable identifying neighborhood characteristics, however, if given more freedom a classification model and no grouping of venues could potentially be a better and more reliable predictor. Additionally, more research into the problem could be given, and sources from said research cited to bolster the legitimacy of the project. There is also a potential for the chosen results for this project to have an equally successful chance as the k-means clusters = 5 outcome, and it is equally important to note that on a given day the results from the Foursquare search could change, completely throwing off the results in this report.
## Conclusion
Using a k-means clustering machine learning algorithm this project identified, using the optimum model run, the Central Park neighborhood in Denver, Colorado as the most suitable neighborhood for stakeholders to open a new brewery based on its similarity to other neighborhoods with brewery being listed in the top 10 most occurring venues. The Montbello neighborhood could also be a potential site but holds less confidence. It is important to note that the results of this model change frequently based on search results from the Foursquare API. It is recommended that future projects should search for other, possibly more reliable, means of neighborhood classification in order to identify potential venue sites.